In [1]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('nlp2').getOrCreate()

In [2]:
from pyspark.ml.feature import Tokenizer, RegexTokenizer, HashingTF, IDF, Tokenizer

In [3]:
sentenceData = spark.createDataFrame([
  (0.0, 'Hi I heard about Spark'),
  (0.0, 'I wish Java could use case classes'),
  (1.0, 'Logistic regression models are neat')
], ['label', 'sentence'])

In [4]:
sentenceData.show()

+-----+--------------------+
label| sentence|
+-----+--------------------+
 0.0|Hi I heard about ...|
 0.0|I wish Java could...|
 1.0|Logistic regressi...|
+-----+--------------------+

In [5]:
tokenizer = Tokenizer(inputCol='sentence', outputCol='words')
words_data = tokenizer.transform(sentenceData)

In [6]:
words_data.show()

+-----+--------------------+--------------------+
label| sentence| words|
+-----+--------------------+--------------------+
 0.0|Hi I heard about ...|[hi, i, heard, ab...|
 0.0|I wish Java could...|[i, wish, java, c...|
 1.0|Logistic regressi...|[logistic, regres...|
+-----+--------------------+--------------------+

In [7]:
hashing_tf = HashingTF(inputCol='words', outputCol='rawFeatures')

In [8]:
featurized_data = hashing_tf.transform(words_data)

In [9]:
tokenizer = Tokenizer(inputCol='sentence', outputCol='words')

In [10]:
words_data = tokenizer.transform(sentenceData)

In [11]:
words_data.show(truncate=False)

+-----+-----------------------------------+------------------------------------------+
label|sentence |words |
+-----+-----------------------------------+------------------------------------------+
0.0 |Hi I heard about Spark |[hi, i, heard, about, spark] |
0.0 |I wish Java could use case classes |[i, wish, java, could, use, case, classes]|
1.0 |Logistic regression models are neat|[logistic, regression, models, are, neat] |
+-----+-----------------------------------+------------------------------------------+

In [12]:
hashing_tf = HashingTF(inputCol='words', outputCol='rawFeatures')

In [13]:
featurized_data = hashing_tf.transform(words_data)

In [14]:
idf = IDF(inputCol='rawFeatures', outputCol='features')

In [15]:
idf_model = idf.fit(featurized_data)

In [16]:
rescaled_data = idf_model.transform(featurized_data)

In [17]:
rescaled_data.select('label', 'features').show(truncate=False)

+-----+------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
label|features |
+-----+------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
0.0 |(262144,[24417,49304,73197,91137,234657],[0.28768207245178085,0.6931471805599453,0.6931471805599453,0.6931471805599453,0.6931471805599453]) |
0.0 |(262144,[20719,24417,55551,116873,147765,162369,192310],[0.6931471805599453,0.28768207245178085,0.6931471805599453,0.6931471805599453,0.6931471805599453,0.6931471805599453,0.6931471805599453])|
1.0 |(262144,[13671,91006,132713,167122,190884],[0.6931471805599453,0.6931471805599453,0.6931471805599453,0.6931471805599453,0.6931471805599453]) |
+-----+------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+

In [18]:
from pyspark.ml.feature import CountVectorizer

In [19]:
df = spark.createDataFrame([
  (0, "a b c".split(" ")),
  (0, "a b b c a".split(" ")),
], ["id", "words"])

In [20]:
df.show()

+---+---------------+
 id| words|
+---+---------------+
 0| [a, b, c]|
 0|[a, b, b, c, a]|
+---+---------------+

In [21]:
# vocabSize=3 because there are 3 distinct words: a,b and c
# minDF=2.0 states the min no. of docs the word must appear in, in order to be included in the count vectorizer
cv = CountVectorizer(inputCol='words', outputCol='features', vocabSize=3, minDF=2.0)

In [22]:
model = cv.fit(df)

In [23]:
result = model.transform(df)

In [24]:
result.show(truncate=False)

+---+---------------+-------------------------+
id |words |features |
+---+---------------+-------------------------+
0 |[a, b, c] |(3,[0,1,2],[1.0,1.0,1.0])|
0 |[a, b, b, c, a]|(3,[0,1,2],[2.0,2.0,1.0])|
+---+---------------+-------------------------+